In [2]:
import forallpeople
forallpeople.environment('Structural',top_level=True)
#Useful extra units
cm = 1e-2*m #type: ignore
from IPython.display import Markdown, display
import handcalcs.render

Calculations
Let the force be represented as:
$$F=ma$$
Where:
- $m$ is the mass in $\text{kg}$
- $a$ is the acceleration in $\text{m/s}^2$

In [10]:
# Example calculation
mass=100*kg
a = 9.81*m/s**2
force = mass * a
force=5*kN
print(a)
b=2.83*cm**2/m
print(b.value*10000)
display(Markdown(f"The result is: $F$={round(force,1)}"))
t = 3.75*mm #Espesor
H = 300*mm #Altura
B = 100*mm #Ancho
I_x = B*H**3/12 - (B - t)*(H - t)**3/12
I_y = H*B**3/12 - (H - t)*(B - t)**3/12
display(Markdown(f"The result is: $I_x$={round(I_x,0)}"))
display(Markdown(f"The result is: $I_y$={round(I_y,0)}"))
# Cambio de unidad
force.to(unit_name="kip")


9.810 m·s⁻²
2.83


The result is: $F$=5.0 kN

The result is: $I_x$=16457503 mm⁴

The result is: $I_y$=2986995 mm⁴

1.124 kip

In [3]:
%%capture
!jupyter nbconvert --to markdown --no-input report_example.ipynb

In [4]:
%%capture
!jupyter nbconvert --no-input report_example.ipynb --to html

[NbConvertApp] Converting notebook report_example.ipynb to html
[NbConvertApp] Writing 280683 bytes to report_example.html


In [ ]:
%%capture
!jupyter nbconvert --to pdf --no-input report_example.ipynb

In [ ]:
%%capture
!jupyter nbconvert --to pdf report_example.ipynb

In [5]:
%%render 2
mass=100*kg
a = 9.81*m/s**2
force = mass * a

<IPython.core.display.Latex object>

In [ ]:
%%render 2
mass=100*kg
a = (9.81*m/s**2)
force = (mass * a)

<IPython.core.display.Latex object>

In [ ]:
%%render params 2 
mass=100*kg
a = 9.81*m/s**2
force = mass * a

<IPython.core.display.Latex object>

In [ ]:
%%render symbolic
mass=100*kg #Comment in line
a = 9.81*m/s**2
force = mass * a


<IPython.core.display.Latex object>

In [ ]:
%%render params 2
force = mass * a

<IPython.core.display.Latex object>

In [ ]:
%%render params 2
#Tubo 200x100x3.75mm
t = 3.75*mm #Espesor
H = 200*mm #Altura
B = 100*mm #Ancho

<IPython.core.display.Latex object>

In [ ]:
%%render
I_x = B*H**3/12 - (B - t)*(H - t)**3/12
I_y = H*B**3/12 - (H - t)*(B - t)**3/12

<IPython.core.display.Latex object>

In [15]:
import sys
import os
# Añadir el directorio raíz al PYTHONPATH
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

from structurelab.settings import Settings
from structurelab.material import create_concrete, SteelBar
from structurelab.section import Beam

from pylatex import Command, Document, Math, Matrix, Section, Subsection, VectorName, NoEscape, Figure, LongTable
from pylatex.utils import NoEscape
import matplotlib
matplotlib.use("Agg")  # Not to use X server. For TravisCI.
import matplotlib.pyplot as plt  # noq

class MyDocument(Document):
    def __init__(self, documentclass='article', document_options=['a4paper','spanish','10pt'], geometry_options={"left": "20mm","right": "20mm"}):
        # Pasar las opciones al constructor de la clase base Document
        super().__init__(documentclass=documentclass, document_options=document_options, geometry_options=geometry_options)
        self.preamble.append(Command("usepackage","mathtools"))
        self.preamble.append(Command("title", "Viga rectangular a flexión - ACI 318-19"))
        self.preamble.append(Command("author", "StructureLab"))
        self.preamble.append(Command("date", NoEscape(r"\today")))
        self.append(NoEscape(r"\maketitle"))


def main():
    # Ejemplo de uso
    concrete=create_concrete(name="H30",f_c=30*MPa, design_code="ACI 318-19") # type: ignore
    steelBar=SteelBar(name="ADN 420", f_y=420*MPa) # type: ignore
    section = Beam(
        name="V-40x50",
        concrete=concrete,
        steelBar=steelBar,
        width=400 * mm,  # type: ignore
        depth=500 * mm,  # type: ignore
    )
    print(f"Nombre de la sección: {section.get_name()}")

    doc = MyDocument()
    doc.append(NoEscape(
        r"""
        La resistencia nominal a flexión, $M_n$, de una viga de hormigón armado se calcula mediante la siguiente expresión:

        \begin{equation}
        M_n = A_s \cdot f_y \cdot \left( d - \frac{a}{2} \right)
        \end{equation}

        Donde:

        \begin{itemize}
            \item $A_s$ es el área de la armadura de refuerzo (en $\text{mm}^2$).
            \item $f_y$ es el esfuerzo de fluencia del acero de refuerzo (en MPa).
            \item $d$ es la distancia desde la fibra extrema en compresión hasta el centroide del acero de refuerzo (en mm).
            \item $a$ es la profundidad del bloque de compresión equivalente (en mm), que se determina como:
            \begin{equation}
            a = \frac{\beta_1 \cdot c}{2}
            \end{equation}
        \end{itemize}

        La profundidad $c$ se relaciona con la deformación en el acero de refuerzo, y $\beta_1$ es un factor que depende de la resistencia especificada del concreto, $f'_c$, según las disposiciones del ACI 318-19.

        \section{Factor de Resistencia}

        Para asegurar un diseño seguro, se introduce un factor de resistencia, $\phi$, que se aplica a la resistencia nominal:

        \begin{equation}
        M_u \leq \phi \cdot M_n
        \end{equation}

        Donde $M_u$ es el momento flector último requerido debido a las cargas aplicadas, y $\phi$ es el factor de reducción de resistencia, cuyo valor depende del tipo de fallo esperado y puede variar entre 0.65 y 0.90.

        """))
    doc.generate_pdf("Pylatex_test",clean_tex=False)

if __name__ == "__main__":
    main()

Nombre de la sección: V-40x50
